In [1]:
import os
import random
import shutil

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import torch
import torchvision
from sklearn.metrics import classification_report, confusion_matrix
from ultralytics import YOLO

In [ ]:

print("PyTorch version:", torch.__version__)
print("torchvision version:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)


PyTorch version: 2.5.1+cu118
torchvision version: 0.20.1+cu118
CUDA available: True
CUDA version: 11.8


In [ ]:
# Set paths for data splitting and preparation
original_dataset_dir = 'Datasets/DBX/Cactus'
output_base_dir = 'DatasetPreped2'
train_dir = os.path.join(output_base_dir, 'train')
val_dir = os.path.join(output_base_dir, 'val')
test_dir = os.path.join(output_base_dir, 'test')


In [ ]:
# Dataset splitting function
def split_dataset_with_health_status(original_dataset_dir, output_base_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42):
    assert train_ratio + val_ratio + test_ratio == 1.0, "Train, val, and test ratios must sum to 1."
    random.seed(seed)

    train_dir = os.path.join(output_base_dir, 'train')
    val_dir = os.path.join(output_base_dir, 'val')
    test_dir = os.path.join(output_base_dir, 'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    diseased_folders = ['Damaged', 'EarlyStage', 'LateStage', 'Old_Dead']
    healthy_folder = 'Healthy'

    # Handling Healthy class
    healthy_path = os.path.join(original_dataset_dir, healthy_folder)
    if os.path.isdir(healthy_path):
        os.makedirs(os.path.join(train_dir, 'Healthy'), exist_ok=True)
        os.makedirs(os.path.join(val_dir, 'Healthy'), exist_ok=True)
        os.makedirs(os.path.join(test_dir, 'Healthy'), exist_ok=True)
        healthy_images = [img for img in os.listdir(healthy_path) if os.path.isfile(os.path.join(healthy_path, img))]
        random.shuffle(healthy_images)
        train_end = int(len(healthy_images) * train_ratio)
        val_end = train_end + int(len(healthy_images) * val_ratio)
        train_images = healthy_images[:train_end]
        val_images = healthy_images[train_end:val_end]
        test_images = healthy_images[val_end:]
        for img in train_images:
            shutil.copy(os.path.join(healthy_path, img), os.path.join(train_dir, 'Healthy', img))
        for img in val_images:
            shutil.copy(os.path.join(healthy_path, img), os.path.join(val_dir, 'Healthy', img))
        for img in test_images:
            shutil.copy(os.path.join(healthy_path, img), os.path.join(test_dir, 'Healthy', img))

    # Handling Diseased class
    os.makedirs(os.path.join(train_dir, 'Diseased'), exist_ok=True)
    os.makedirs(os.path.join(val_dir, 'Diseased'), exist_ok=True)
    os.makedirs(os.path.join(test_dir, 'Diseased'), exist_ok=True)
    for diseased_subfolder in diseased_folders:
        diseased_path = os.path.join(original_dataset_dir, diseased_subfolder)
        if os.path.isdir(diseased_path):
            diseased_images = [img for img in os.listdir(diseased_path) if os.path.isfile(os.path.join(diseased_path, img))]
            random.shuffle(diseased_images)
            train_end = int(len(diseased_images) * train_ratio)
            val_end = train_end + int(len(diseased_images) * val_ratio)
            train_images = diseased_images[:train_end]
            val_images = diseased_images[train_end:val_end]
            test_images = diseased_images[val_end:]
            for img in train_images:
                shutil.copy(os.path.join(diseased_path, img), os.path.join(train_dir, 'Diseased', img))
            for img in val_images:
                shutil.copy(os.path.join(diseased_path, img), os.path.join(val_dir, 'Diseased', img))
            for img in test_images:
                shutil.copy(os.path.join(diseased_path, img), os.path.join(test_dir, 'Diseased', img))

In [ ]:
#split_dataset_with_health_status(original_dataset_dir, output_base_dir)

In [ ]:
# Base directories
base_dir = 'DatasetPreped2'
images_dir = os.path.join(base_dir, 'images')
annotations_dir = os.path.join(base_dir, 'labels')


# Class mapping
class_mapping = {
    'Healthy': 0,
    'Diseased': 1
}

In [ ]:
# Iterate through train, val, and test folders
for split in ['train', 'val', 'test']:
    split_dir = os.path.join(images_dir, split)
    for class_name, class_id in class_mapping.items():
        class_dir = os.path.join(split_dir, class_name)
        if os.path.isdir(class_dir):
            # Create corresponding annotation directory if not exists
            annotation_split_dir = os.path.join(annotations_dir, split, class_name)
            os.makedirs(annotation_split_dir, exist_ok=True)

            # Process each image in the class directory
            for image_name in os.listdir(class_dir):
                if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):  # Filter image files
                    # Create a corresponding .txt file for annotations
                    annotation_file_path = os.path.join(annotation_split_dir, image_name.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt'))

                    # Write bounding box data for the whole image
                    with open(annotation_file_path, 'w') as f:
                        # YOLO format: class_id x_center y_center width height
                        f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")

print("Individual YOLO annotation files created successfully.")

Individual YOLO annotation files created successfully.


In [ ]:
# Define base directory paths
images_dir = 'images'

# Create the content for data.yaml for object detection
data_yaml_content = f"""
train: {os.path.join(images_dir, 'train')}
val: {os.path.join(images_dir, 'val')}
test: {os.path.join(images_dir, 'test')}  # Optional, include if test set is available

nc: 2  # Number of classes

names: ['Healthy', 'Diseased']  # Class names
"""

# Write the content to a data.yaml file
output_yaml_path = os.path.join(base_dir, 'data.yaml')
with open(output_yaml_path, 'w') as yaml_file:
    yaml_file.write(data_yaml_content)

print(f"data.yaml file created at: {output_yaml_path}")

data.yaml file created at: DatasetPreped2\data.yaml


In [ ]:
!nvidia-smi

Tue Nov 12 11:55:48 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.03                 Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   53C    P8              3W /   96W |     321MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from ultralytics import YOLO

# Load YOLOv8 model for object detection
model = YOLO('yolov8n.pt')  # Use 'yolov8n.pt' for a smaller model; other variants like 'yolov8s.pt' are available for larger models


In [ ]:
from ultralytics import YOLO

# Load YOLOv8 model for object detection
model = YOLO('yolov8n.pt')

# Train the model with custom data augmentations
model.train(
    data='DatasetPreped2/data.yaml',
    epochs=20,
    imgsz=256,
    batch=16,
    augment=True,  # Enable data augmentation
    hsv_h=0.015,  # Hue variation
    hsv_s=0.7,    # Saturation variation
    hsv_v=0.4,    # Brightness variation
    fliplr=0.5,   # Horizontal flip
    scale=0.5,    # Scale images
    translate=0.1,  # Translate images
    mosaic=1.0,    # Mosaic augmentation
    mixup=0.0,     # Mixup augmentation
    val=True,  # Enable validation tracking
    save=True,  # Save checkpoints
    save_period=1  # Save after every epoch
)


Ultralytics 8.3.29  Python-3.10.11 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=DatasetPreped2/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

train: Scanning C:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\DatasetPreped2\labels\train\Diseased.cache... 2275 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2275/2275 [00:00<?, ?it/s]
val: Scanning C:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\DatasetPreped2\labels\val\Diseased.cache... 486 images, 0 backgrounds, 0 corrupt: 100%|██████████| 486/486 [00:00<?, ?it/s]


Plotting labels to C:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\datasets\runs\detect\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to C:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\datasets\runs\detect\train4
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20     0.457G     0.3788      1.198      1.036         10        256: 100%|██████████| 143/143 [00:09<00:00, 14.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:01<00:00, 10.89it/s]

                   all        486        486      0.974      0.491      0.505      0.501


...

                   all        486        486      0.518          1      0.551       0.55



20 epochs completed in 0.059 hours.
Optimizer stripped from C:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\datasets\runs\detect\train4\weights\last.pt, 6.2MB
Optimizer stripped from C:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\datasets\runs\detect\train4\weights\best.pt, 6.2MB

Validating C:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\datasets\runs\detect\train4\weights\best.pt...
Ultralytics 8.3.29  Python-3.10.11 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:01<00:00,  8.50it/s]


                   all        486        486       0.47          1      0.537      0.537
               Healthy         18         18      0.037          1      0.094      0.094
              Diseased        468        468      0.903          1       0.98       0.98
Speed: 0.0ms preprocess, 1.5ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to C:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\datasets\runs\detect\train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001CA2841EA40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [ ]:
# Load the best model for testing
best_model_path = 'datasets/runs/detect/train4/weights/last.pt'
model_test = YOLO(best_model_path)

In [ ]:
# Define paths
test_data_path = 'DatasetPreped2/images/test'
subfolders = ['Diseased', 'Healthy']
true_labels = {'Diseased': 1, 'Healthy': 0}  # Adjust class mapping as per your YOLO class indices

# Placeholder lists for true and predicted labels
y_true = []
y_pred = []

In [ ]:
# Iterate over all images in both subfolders
for subfolder in subfolders:
    folder_path = os.path.join(test_data_path, subfolder)
    images = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    for image_name in images:
        image_path = os.path.join(folder_path, image_name)
        
        # Run YOLO prediction
        results = model.predict(source=image_path, classes=[0, 1])  # Only predict on classes 0 and 1


        # Assume the model predicts the highest confidence class; extract the predicted label
        if results and results[0].boxes.cls is not None:
            predicted_label = int(results[0].boxes.cls[0])  # Predicted class index
        else:
            predicted_label = -1  # Use -1 to represent no prediction
        
        # Append true and predicted labels
        y_true.append(true_labels[subfolder])  # True label inferred from the folder name
        y_pred.append(predicted_label)


image 1/1 c:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\DatasetPreped2\images\test\Diseased\0002e4f0-cefb-4721-be48-aa9566b823ea_256_512_512_768.png: 256x256 1 Diseased, 36.0ms
Speed: 1.0ms preprocess, 36.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 c:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\DatasetPreped2\images\test\Diseased\08734f1f-e770-48c0-87ee-89e06e3600a7_512_768_256_512.png: 224x256 1 Diseased, 31.6ms
Speed: 1.5ms preprocess, 31.6ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 256)

image 1/1 c:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\DatasetPreped2\images\test\Diseased\0c98e120-07a4-4a55-ae17-bb8bb32d4742_512_768_0_256.png: 256x256 1 Diseased, 33.0ms
Speed: 1.0ms preprocess, 33.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 c:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\DatasetPreped2\images\test\Diseased\0c98e120-07a4-4a5

In [ ]:
# Generate and display confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
class_names = ['Healthy', 'Diseased']  # Adjust to match your dataset classes
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Print a detailed classification report
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=class_names))

<Figure size 800x600 with 2 Axes>

Classification Report:
               precision    recall  f1-score   support

     Healthy       0.00      0.00      0.00        19
    Diseased       0.96      1.00      0.98       472

    accuracy                           0.96       491
   macro avg       0.48      0.50      0.49       491
weighted avg       0.92      0.96      0.94       491



c:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\khali\OneDrive\Bureau\amine\CactiViT-materials-master\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

In [ ]:
!pip freeze > requirements.txt